In [8]:
%matplotlib qt5
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

from Tkinter import Tk
import tkFileDialog as filedialog

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual

pp = pprint.PrettyPrinter(indent=4)

In [9]:
def uvMapping(_img, get_overlay=True,verbose=False):
    overlay = None
    h,w = _img.shape
    histRange = (0,256)
    histSz = np.max(_img) + 1
    if(verbose): print("[UV Mapping] Input Image Size: (%d, %d)" % (h,w))

    umap = np.zeros((histSz,w,1), dtype=np.uint8)
    vmap = np.zeros((h,histSz,1), dtype=np.uint8)

    for i in range(0,h):
        vscan = _img[i,:]
        vrow = cv2.calcHist([vscan],[0],None,[histSz],histRange)
        if(verbose): print("\t[V Mapping] Scan [%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, vscan.shape)), ', '.join(map(str, vrow.shape))))
        vmap[i,:] = vrow

    for i in range(0,w):
        uscan = _img[:,i]
        urow = cv2.calcHist([uscan],[0],None,[histSz],histRange)
        if(verbose): print("\t[U Mapping] Scan[%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, uscan.shape)), ', '.join(map(str, urow.shape))))
        umap[:,i] = urow

    umap = np.reshape(umap,(histSz,w))
    vmap = np.reshape(vmap,(h,histSz))

    if(get_overlay):
        blank = np.ones((umap.shape[0],vmap.shape[1]),np.uint8)*255
        pt1 = np.concatenate((_img, vmap), axis=1)
        pt2 = np.concatenate((umap,blank),axis=1)
        overlay = np.concatenate((pt1,pt2),axis=0)
        overlay = cv2.cvtColor(overlay,cv2.COLOR_GRAY2BGR)

    if(verbose):
        print("\t[UV Mapping] U Map = (%s) ----- V Map = (%s)" % (', '.join(map(str, umap.shape)),', '.join(map(str, vmap.shape)) ))
    return umap,vmap,overlay

In [10]:
def construct_helper_img(_imgs):
    print(_imgs[0].shape)
    n,m = _imgs[0].shape[0], _imgs[0].shape[1]
    print(n,m)
    
    bborder = np.ones((5,m,3))*(255,0,0)
    sborder = np.ones((n,5,3))*(255,0,0)
    cborder = np.ones((5,5,3))*(255,0,0)
    
    helper_img_t = np.concatenate(
        (
            np.concatenate((cv2.cvtColor(_imgs[0],cv2.COLOR_GRAY2BGR),sborder), axis=1),
            np.concatenate((_imgs[1],sborder), axis=1),
            np.concatenate((cv2.cvtColor(_imgs[2],cv2.COLOR_GRAY2BGR),sborder), axis=1)
        ), axis=1
    )
    
    helper_img_m = np.concatenate(
        (
            np.concatenate((bborder,cborder), axis=1),
            np.concatenate((bborder,cborder), axis=1),
            np.concatenate((bborder,cborder), axis=1)
        ), axis=1
    )
    
    helper_img_b = np.concatenate(
        (
            np.concatenate((cv2.cvtColor(_imgs[3],cv2.COLOR_GRAY2BGR),sborder), axis=1),
            np.concatenate((cv2.cvtColor(_imgs[4],cv2.COLOR_GRAY2BGR),sborder), axis=1),
            np.concatenate((cv2.cvtColor(_imgs[5],cv2.COLOR_GRAY2BGR),sborder), axis=1)
        ), axis=1
    )
    
    helper_img = np.concatenate((helper_img_t,helper_img_m,helper_img_b), axis=0)
    return helper_img




    



In [11]:
def line_finder(_img,method=1,line_input_method=0,e1=2,e2=5,ang=90,rho=1,minLineLength=11,maxLineGap=11,
                houghThresh=50,greyThresh=11,show_helpers=True,use_umap=True,flip_thresh_bin=False):
    global flag_new_img, last_img, umap, vmap, overlay, helper_img,prev_e1,prev_e2,prev_greyThresh, prev_map
    
    if(last_img == _img):
        flag_new_img = False
        print("No new Image")
    else:
        flag_new_img = True
        print("New Image!!!")
        
    if(prev_map == use_umap): flg1 = False
    else: flg1 = True
      
    if(prev_greyThresh == greyThresh): flg2 = False
    else: flg2 = True
        
    if(prev_e1 == e1): flg3 = False
    else: flg3 = True

    if(prev_e2 == e2): flg4 = False
    else: flg4 = True
    
    if(flg1 or flg2 or flg3 or flg4):
        flag_needs_filtering = True
        print("Images need to be filtered again...")
    else:
        flag_needs_filtering = True
        print("Skipping image filtering...")
    
    last_img = _img
    prev_e1 , prev_e2 = e1, e2
    prev_greyThresh = greyThresh
    prev_map = use_umap
    
    print(
    """
    Inputs:
    ------
     
    \t* Kernel Size       : (%d, %d)
    \t* Rho, Angle (deg)  :  %d, %d
    \t* Min Line Length   :  %d
    \t* Max Line Gap      :  %d
    \t* Grey Thresholding :  %d
    \t* Hough Threshold   :  %d
    """ % (
        e1,e2,rho,ang,minLineLength,maxLineGap,greyThresh,houghThresh
    ))
    
    # Convert Angle to radians for calculations
    ang = ang * np.pi/180
    
    if(flag_new_img):
        img = cv2.imread(_img,cv2.IMREAD_GRAYSCALE)
        print("Mapping new image into UV Map")
        umap,vmap, overlay = uvMapping(img)
    
    if(use_umap): tmpIn = np.copy(umap)
    else: tmpIn = np.copy(vmap)

    tmp = cv2.cvtColor(tmpIn,cv2.COLOR_GRAY2BGR)
    n,m,_ = tmp.shape
    filtered = np.zeros((n,m,3))

    if(flag_needs_filtering):
        if(flip_thresh_bin): masking = cv2.THRESH_BINARY_INV
        else: masking = cv2.THRESH_BINARY
            
        kernel = np.ones((e1,e2),np.uint8)

        ret, grey = cv2.threshold(tmpIn,greyThresh,255,masking)
        closing = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,kernel, iterations = 2)

        ret, grey_thresh = cv2.threshold(grey,greyThresh,255,masking)
        ret, close_thresh = cv2.threshold(closing,greyThresh,255,masking)
        canny = cv2.Canny(closing,25,200,apertureSize = 3)
        helper_imgs = [tmpIn,tmp,grey,closing,close_thresh,canny]
        helper_img = construct_helper_img(helper_imgs)
    
    if(line_input_method is 1): hlines = closing
    elif(line_input_method is 2): hlines = canny
    else: hlines = grey
    
    try:
        if(method==0):
            lines = cv2.HoughLines(hlines,rho,ang,houghThresh)
            count = 0
            for rho,theta in lines[0]:
                count+=1
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))

                cv2.line(tmp,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.line(filtered,(x1,y1),(x2,y2),(255,0,255),2)
        else:
            lines = cv2.HoughLinesP(hlines,rho,ang,houghThresh,minLineLength,maxLineGap)
            for x in range(0, len(lines)):
                for x1,y1,x2,y2 in lines[x]:
                    cv2.line(tmp,(x1,y1),(x2,y2),(0,255,0),2)
                    cv2.line(filtered,(x1,y1),(x2,y2),(255,0,255),2)
    except:
        print("Couldn't Find Hough Lines!!!")
        pass
    
    if(flag_new_img):
        plt.figure(0)
        plt.imshow(overlay)
        plt.show()

    if(flag_needs_filtering):
        plt.figure(1)
        if(show_helpers):
            plt.imshow(helper_img)
            plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
        plt.show()

    try:
        plt.figure(2)
        plt.imshow(tmp)
        plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
        plt.show()
    except:
        print("Couldn't plot Hough Lines Composite")
        pass

    print(" ============  Plotting Done !! =================")

In [12]:
plt.close('all')
flag_new_img = False
last_img = None
umap = None
vmap = None
overlay = None
helper_img = None

prev_e1,prev_e2,prev_greyThresh = 0, 0, 0
prev_map = False



# =========================================================
#
# =========================================================
p=interactive(line_finder,
    _img="scripts/python/vision_playground/test_disparity.png",
    method={'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    line_input_method={'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1=(1,255,1), e2=(1,255,1),
    ang=(0,360,1), rho=(1,1000,1),
    minLineLength=(0,255,1), maxLineGap=(0,255,1),
    houghThresh=(0,255,1), greyThresh=(0,255,1),
    show_helpers=True,use_umap=True,flip_thresh_bin=False
)
display(p)


aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSdzY3JpcHRzL3B5dGhvbi92aXNpb25fcGxheWdyb3VuZC90ZXN0X2Rpc3Bhcml0eS5wbmcnLCBkZXNjcmlwdGlvbj11J19pbWcnKSzigKY=
